# Project Overview

In this project, we will build a market simulator that accepts trading orders while monitoring the value and performance of the portfolio.

## Our Imports

In [1]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

## Trade History

Since our current portfolio is empty since no orders have been processed (and because the functionality is not complete), we should establish the data type or even just the structure of how we want our trades recorded. I've been playing with panda dataframes lately so let's go down that route.

The columns represent the following:
-

In [2]:
tradeHistoryColumns = ['Date','Symbol','Order','OrderInt','Shares','Price','Value']
tradeHistory = pd.DataFrame(columns=tradeHistoryColumns)

In [3]:
tradeHistory

,Date,Symbol,Order,OrderInt,Shares,Price,Value


## Order Processing

Now when it comes to executing orders, it is a tri-part problem. We first need to fetch the price of a chosen stock. We then have to decide if this is an acceptable price to purchase or sell the stock. Finally, we have to execute this order. So let's begin.

### Fetch Stock Price

Let's leverage the yfinance python package to fetch a stock's particular price. For simplicity purposes, we will be getting the price of a chosen stock from it's adjusted close price for the day. In the future, I'm sure functionality can be extended to be more precise and allow for better stock price selection throughout the day.

In [6]:
def fetchPrice(stockChoices,inputDate):
    priceData = yf.download(stockChoices, start=inputDate, end=inputDate)
    return priceData
    
stockChoices = ['AAPL','GOOGL']

In [7]:
stockPrices = (fetchPrice(stockChoices,"2019-05-29")['Adj Close'])

[*********************100%***********************]  2 of 2 downloaded


Now that we have the the stockPrices in a nice pandas dataframe, all we have to do to filter it is the below.

In [8]:
for x in range(len(stockChoices)):
    print (stockPrices[stockChoices[x]][0])

177.38
1119.94


### Verify Order Exection

Here we want to decide if the price is an acceptable price to purchase/sell the stock. As well as how many units we want to purchase or sell. For now, we will have this as a manual method exercised by the user with functionality added to extend this to apply machine learning techniques to optimize it.

In [9]:
def decideOrder(stockPrice,stockUnits,tradeChoice,userControl=False):
    if (userControl == True):
        return (True,tradeChoice)
    else:
        return False 

In [10]:
decideOrder(100,1,"BUY",True)

(True, 'BUY')

### Update Trades

Now if the we decicde to execute the order, we will first need a function to update our trade history to keep a records of all transactions. So here we want to work on an update trades function to reflect our executed orders impacts on our portfolio. Just a simple append of the new data we want to add to our dataframe.

In [11]:
def updateTrades(tradeHistory,addData):
    updatedHistory = tradeHistory.append(addData)
    return updatedHistory

### Execute Order

Here we will actually execute the order and update our portfolio and our current holdings. So let's work on an execute order function and a method to update our portfolio.

Our execute order function takes in the following as input:
- stockChoices: Which stocks are to be part of this order
- stockUnits: How many of these stocks are to be traded
- inputDate: For which date is this order being made
- currPortfolio: Our currentPorfolio
- userControl: Whether to allow the user to execute or the machine learning model
- tradeChoice: Whether to buy, sell or hold an order

It will then return the boolean of the function and the updated portfolio.

In [12]:
def execOrder(stockChoices,stockUnits,inputDate,currHistory,tradeChoice,userControl=False):
    stockPrices = (fetchPrice(stockChoices,inputDate)['Adj Close'])
    orderDecision = decideOrder(stockPrices,stockUnits,tradeChoice,userControl)
    
    if (orderDecision[0] == True):
        for x in range(len(stockChoices)):
            tradeChoice = -1 if orderDecision[1] == "BUY" else 1
            totalVal = stockUnits[x] * stockPrices[stockChoices[x]][0] * tradeChoice
            addData = [(inputDate,stockChoices[x],orderDecision[1],(tradeChoice*-1),
                        stockUnits[x],stockPrices[stockChoices[x]][0], totalVal)]
            addDataFrame = pd.DataFrame(addData,columns=currHistory.columns)
            currHistory = updateTrades(currHistory,addDataFrame)
        return (True,currHistory)
    else:
        return (False,currHistory)

### Get Portfolio Details

Now if the order executed, I'd like to see my portfolio value. So let's create a display portfolio function to see my current portfolio value. An important consideration to remember is the fact that we may have multiple orders for the same stock. So we have to tally up and do a calculation on how many shares we still own and how much they're worth. 

In [13]:
def getPortfolioValue(currHistory):
    
    stocksHeld = currTrades[1].Symbol.unique()

    portfolioVal = 0
    for x in stocksHeld:
        newDF = currTrades[1].loc[currTrades[1]['Symbol'] == x]
        portfolioVal += newDF.Value.sum()
        
    return abs(portfolioVal)

We also want to do a calculation of how many shares we hold for any stock. This is calculated through the BUY and SELL orders placed.

In [21]:
def getUnitsHeld(currHistory):
    stocksHeld = currTrades[1].Symbol.unique()

    unitsHeld = []
    for x in stocksHeld:
        newDF = currTrades[1].loc[currTrades[1]['Symbol'] == x]
        unitsHeld.append((newDF.Shares * newDF.OrderInt).sum())

    return unitsHeld

We should also have a function to get our starting investment to compare to the current portfolio value. This should be straightforward in the sense we find the very first order executed and calculate how much was invested from the number of shares and share price.

In [42]:
def getStartInvestment(currHistory):
    checkDate = min(currHistory['Date']) 
    newDF = currHistory.loc[currHistory['Date'] == checkDate]
    iniitalInvestment = newDF.Value.sum()
    return abs(iniitalInvestment)

In [38]:
def displayPortfolio(currHistory):
    totalValue = getPortfolioValue(currHistory)
    unitsHeld = getUnitsHeld(currHistory)
    startValue = getStartInvestment(currHistory)
    
    print ("Welcome To Your Portfolio!\n")
    print ("Current holdings include: {0}".format(currHistory.Symbol.unique()))
    print ("Current units held are: {0}".format(unitsHeld))
    print ("Your initial investment was: {0}".format(startValue))
    print ("Total Portfolio Value: {0}".format(totalValue))

## Sample Test

Now let's run a sample to test all this together.

In [17]:
currTrades = execOrder(['AAPL','GOOGL'],[10,10],"2019-05-01",tradeHistory,tradeChoice="BUY",userControl=True)
currTrades = execOrder(['AAPL','GOOGL'],[5,5],"2019-05-02",currTrades[1],tradeChoice="SELL",userControl=True)

[*********************100%***********************]  2 of 2 downloaded
[*********************100%***********************]  2 of 2 downloaded


In [43]:
if (currTrades[0] == True):
    print ("Executed Order!\n")
    print ("Here is your trade history:\n")
    print (currTrades[1])
    print ("\n")
    displayPortfolio(currTrades[1])
else:
    print ("Order did not execute!\n")
    print ("Here is your trade history:\n")
    print (currTrades[1])

Executed Order!

Here is your trade history:

         Date Symbol Order OrderInt Shares    Price     Value
0  2019-05-01   AAPL   BUY        1     10   209.71  -2097.10
0  2019-05-01  GOOGL   BUY        1     10  1173.32 -11733.20
0  2019-05-02   AAPL  SELL       -1      5   208.35   1041.75
0  2019-05-02  GOOGL  SELL       -1      5  1166.51   5832.55


Welcome To Your Portfolio!

Current holdings include: ['AAPL' 'GOOGL']
Current units held are: [5, 5]
Your initial investment was: 13830.3
Total Portfolio Value: 6955.999999999998


## Daily Portfolio Value

Now we want to see our portfolio value over a given time frame. So let's work on a function to do just that. This will leverage our portfolio calculation function that we worked on in the portfolio evaluator and optimizer projects. By default our start date is the date of the first order executed. 

In [56]:
def dailyPortfolioValue(currPortfolio,endDate,startDate=(currTrades[1].Date.tolist())[0]):
    dailyValues = yf.download(currPortfolio[1].Symbol.tolist(), start=startDate, end=endDate)
    normValues = dailyValues['Adj Close'] / dailyValues['Adj Close'].iloc[0]
    allocedValues = normValues * currPortfolio[1].Shares.tolist()
    posVals = allocedValues * getStartInvestment(currPortfolio)
    portVal = pd.DataFrame(get_portfolio_value(pos_vals),columns=['Value'])
    return portVal

Now let's test it out to see what our daily portfolio value is from the first order date to present day.

In [ ]:
portfolioValue = dailyPortfolioValue(currTrades,'2019-05-29')

## Assess Portfolio Performance

We also want to see metrics relating to our portfolio in terms of perforamnce. So we will leverage our portfolio performance function from the previous notebooks below. We will compare our portfolio's performance to the SPX.

In [ ]:
def get_daily_returns(inputStock):
    """Compute and return the daily return values."""
    dr = inputStock.copy()
    dr[1:] = (inputStock[1:] / inputStock[:-1].values) - 1
    dr.iloc[0] = 0 #start daily return counter at 0
    return dr

print ("The daily average return is {0}".format(get_daily_returns(MSFT[0]['5. adjusted close']).mean()))

In [ ]:
def cumReturn(inputStock):
    stockClose = pd.Series(inputStock)
    returns = stockClose / stockClose.shift(1) - 1
    newReturns = (returns + 1).cumprod()[len(returns)-1] - 1
    
    return newReturns

print ("Cumualtive return for MSFT is {0}%".format(cumReturn(MSFT[0]['5. adjusted close'])*100))

In [ ]:
print("The Standard Deviation of Daily Returns is {0}.".format(dr.std()))

In [ ]:
def sharpe_ratio(inputStock):
    
    dailyReturn =  get_daily_returns(MSFT[0]['5. adjusted close'])
    sr = np.sqrt(100) * (dailyReturn.mean() / dailyReturn.std())
    return sr

print("The daily sharpe_ratio is {0}.".format(sharpe_ratio(MSFT[0]['5. adjusted close'])))

In [ ]:
def portfolioPerformance(currPortfolio):
    print ("Date Range: {0} to {1}\n".format())
    
    print ("Sharpe Ratio of Portfolio: {0}\n".format())
    print ("Sharpe Ratio of $SPX: {0}".format())
    
    print ("Cumulative Return of Portfolio: {0}\n".format())
    print ("Cumulative Return of $SPX: {0}".format())
    
    print ("Standard Deviation of Portfolio: {0}\n".format())
    print ("Standard Deviation of $SPX: {0}".format())
    
    print ("Average Daily Return of Portfolio: {0}\n".format())
    print ("Average Daily Return of $SPX: {0}".format())
    
    print ("Final Value of Portfolio: {0}\n".format())
    

## Implementing Order Impact

In this section, how orders impact the market by using random orders to drive the price up/down. 

## Advancing Order Processing

Now let's add functionality to allow for market, limit, stop orders and maybe even short positions. This will also implement a cash balance and having orders either add or take away from this balance.

## Accounting for All Costs 

In this section we will tackle the real world costs associated with any order. We'll account for commission against the cash balance for each order executed. As well as scenarios for the different type of order types.